In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas
import IPython.display

import raha
import datetime

In [3]:
unary_fd_example = {
    "name": "binary_fd_example",
    "path": "../datasets/unary-fd-example/dirty.csv",
    "clean_path": "../datasets/unary-fd-example/clean.csv"
}

binary_fd_example = {
    "name": "binary_fd_example",
    "path": "../datasets/binary-fd-example/dirty.csv",
    "clean_path": "../datasets/binary-fd-example/clean.csv"
}

# Unary FDs
Im folgenden Beispiel zeige ich, wie die unary FD - Implementierung in Baran funktioniert. Dazu stelle ich alle anderen Featuregeneratoren mittels `VICINITY_ONLY = True` aus. Der Metalearner ist ein Decision Tree Classifier, und das labeling-budget limitiere ich auf 1 -- damit zwinge ich den Metalearner, mit einem test-set von einem Tupel auszukommen. Somit ist es zwingend notwendig, dass für eine korrekte Datensäuberung die Features eine hohe Qualität haben.

In [4]:
datasets = [unary_fd_example]

for dataset_dictionary in datasets:
    app_2 = raha.Correction()

    app_2.LABELING_BUDGET = 1
    app_2.VERBOSE = True
    app_2.CLASSIFICATION_MODEL = "DTC"
    app_2.EXPERIMENT = 'adder'
    app_2.VICINITY_ONLY = True

    d = raha.dataset.Dataset(dataset_dictionary, n_rows=100)
    d.detected_cells = dict(d.get_actual_errors_dictionary())
    d = app_2.initialize_dataset(d)

    app_2.initialize_models(d)
    while len(d.labeled_tuples) < app_2.LABELING_BUDGET:
        app_2.sample_tuple(d, random_seed=0)
        if d.has_ground_truth:
            app_2.label_with_ground_truth(d)
        app_2.update_models(d)
        app_2.generate_features(d)
        app_2.predict_corrections(d, random_seed=0)

    p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
    print({'dataset': d.name, 'experiment': app_2.EXPERIMENT, 'precision': p, 'recall': r, 'f1': f})

The error corrector models are initialized.
Tuple 3 is sampled.
Tuple 3 is labeled.
The error corrector models are updated with new labeled tuple 3.
9 pairs of (a data error, a potential correction) are featurized.
100% (3 / 3) of data errors are corrected.
{'dataset': 'binary_fd_example', 'experiment': 'adder', 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}


Mit nur einem korrekten Beispiel in den ungesäuberten Daten und einem Label ist Baran in der Lage, zwei falsche ZIP-Codes für St. Petersburg zu reinigen.

In [5]:
d.dataframe

,Unnamed: 0,name,zip,country,city
0,0,A,1,Russia,St. Petersburg
1,1,B,123456,Russia,St. Petersburg
2,2,C,123456,Russia,St. Petersburg
3,3,D,123456,Russia,St. Petersburg
4,4,E,2,Russia,Moscow
5,5,F,3,Russia,Vladivostok
6,6,G,10,USA,Miami
7,7,H,10,USA,Miami
8,8,I,10,USA,Miami
9,9,J,100,Germany,Potsdam


Erkannt wird, dass `city -> zip` gilt. Diese unäre Dependency muss sich intern durchsetzen gegen anderen Features. Schauen wir uns das an:

In [6]:
d.pair_features

{(1, 2): {'1': array([0. , 0. , 0. , 0.5, 1. ]),
  '2': array([0.  , 0.  , 0.  , 0.25, 0.  ]),
  '3': array([0.  , 0.  , 0.  , 0.25, 0.  ])},
 (2, 2): {'1': array([0. , 0. , 0. , 0.5, 1. ]),
  '2': array([0.  , 0.  , 0.  , 0.25, 0.  ]),
  '3': array([0.  , 0.  , 0.  , 0.25, 0.  ])},
 (3, 2): {'1': array([1. , 1. , 0. , 0.5, 1. ]),
  '2': array([0.  , 0.  , 0.  , 0.25, 0.  ]),
  '3': array([0.  , 0.  , 0.  , 0.25, 0.  ])}}

Hier setzt sich `city -> zip` gegen `country -> zip` durch. Das geschieht nur, wenn es genügend Beispiele für diese Beziehung gibt. Löscht man die Zeile mit `city = Vladivostok` aus der Tablle, funktioniert die das Adversarial Example nicht mehr. Das demonstriere ich an dieser Stelle aber nicht.

## Binary FDs

Im Folgenden beißt sich Barans FD-Implementierung die Zähne aus.

In [8]:
datasets = [binary_fd_example]

for dataset_dictionary in datasets:
    app_2 = raha.Correction()

    app_2.LABELING_BUDGET = 2
    app_2.VERBOSE = True
    app_2.CLASSIFICATION_MODEL = "DTC"
    app_2.EXPERIMENT = 'adder'
    app_2.VICINITY_ONLY = True
    app_2.SECOND_ORDER_VICINITY = False

    d = raha.dataset.Dataset(dataset_dictionary, n_rows=100)
    d.detected_cells = dict(d.get_actual_errors_dictionary())
    d = app_2.initialize_dataset(d)

    app_2.initialize_models(d)
    while len(d.labeled_tuples) < app_2.LABELING_BUDGET:
        app_2.sample_tuple(d, random_seed=0)
        if d.has_ground_truth:
            app_2.label_with_ground_truth(d)
        app_2.update_models(d)
        app_2.generate_features(d)
        app_2.predict_corrections(d, random_seed=0)

    p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
    print({'dataset': d.name, 'experiment': app_2.EXPERIMENT, 'precision': p, 'recall': r, 'f1': f})

The error corrector models are initialized.
Tuple 8 is sampled.
Tuple 8 is labeled.
The error corrector models are updated with new labeled tuple 8.
18 pairs of (a data error, a potential correction) are featurized.
20% (1 / 5) of data errors are corrected.
Tuple 7 is sampled.
Tuple 7 is labeled.
The error corrector models are updated with new labeled tuple 7.
18 pairs of (a data error, a potential correction) are featurized.
40% (2 / 5) of data errors are corrected.
{'dataset': 'binary_fd_example', 'experiment': 'adder', 'precision': 1.0, 'recall': 0.4, 'f1': 0.5714285714285715}


In [33]:
d.create_repaired_dataset(d.corrected_cells)
d.repaired_dataframe

,Unnamed: 0,name,zip,country,city
0,0,A,1,Russia,St. Petersburg
1,1,B,123456,Russia,St. Petersburg
2,2,C,123456,Russia,St. Petersburg
3,3,D,123456,Russia,St. Petersburg
4,4,E,2,Russia,Moscow
5,5,F,3,Russia,Vladivostok
6,6,G,10,USA,St. Petersburg
7,7,H,10,USA,St. Petersburg
8,8,I,10,USA,St. Petersburg
9,9,J,100,Germany,Potsdam


Man kann sehr schön mit dem `random_seed` des tuple-sampling spielen. Ist diese 0, werden die beiden St. Petersburgs, USA, gesampelt und korrekt gesäubert. Die Information reicht aber nicht aus, um St. Petersburg, Russia korrekt zu reinigen.

Mit `random_seed=1` erhält man eine Auswahl, bei dem die beiden St. Petersburgs, Russia, gelabelt werden. Diese Information reicht dann im Weiteren nicht aus, um die St. Petersburgs, USA zu reinigen.

## Binäre FDs
Das Ergebnis sieht erheblich anders aus, wenn ich FDs 2. Ordnung benutze beim Feature-Sampling.

In [9]:
datasets = [binary_fd_example]

for dataset_dictionary in datasets:
    app_2 = raha.Correction()

    app_2.LABELING_BUDGET = 2
    app_2.VERBOSE = True
    app_2.CLASSIFICATION_MODEL = "DTC"
    app_2.EXPERIMENT = 'adder'
    app_2.VICINITY_ONLY = True
    app_2.SECOND_ORDER_VICINITY = True

    d = raha.dataset.Dataset(dataset_dictionary, n_rows=100)
    d.detected_cells = dict(d.get_actual_errors_dictionary())
    d = app_2.initialize_dataset(d)

    app_2.initialize_models(d)
    while len(d.labeled_tuples) < app_2.LABELING_BUDGET:
        app_2.sample_tuple(d, random_seed=0)
        if d.has_ground_truth:
            app_2.label_with_ground_truth(d)
        app_2.update_models(d)
        app_2.generate_features(d)
        app_2.predict_corrections(d, random_seed=0)

    p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
    print({'dataset': d.name, 'experiment': app_2.EXPERIMENT, 'precision': p, 'recall': r, 'f1': f})

The error corrector models are initialized.
Tuple 8 is sampled.
Tuple 8 is labeled.
The error corrector models are updated with new labeled tuple 8.
18 pairs of (a data error, a potential correction) are featurized.
100% (5 / 5) of data errors are corrected.
Tuple 11 is sampled.
Tuple 11 is labeled.
The error corrector models are updated with new labeled tuple 11.
18 pairs of (a data error, a potential correction) are featurized.
100% (5 / 5) of data errors are corrected.
{'dataset': 'binary_fd_example', 'experiment': 'adder', 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}


In [10]:
d.create_repaired_dataset(d.corrected_cells)
d.repaired_dataframe

,Unnamed: 0,name,zip,country,city
0,0,A,1,Russia,St. Petersburg
1,1,B,1,Russia,St. Petersburg
2,2,C,1,Russia,St. Petersburg
3,3,D,1,Russia,St. Petersburg
4,4,E,2,Russia,Moscow
5,5,F,3,Russia,Vladivostok
6,6,G,10,USA,St. Petersburg
7,7,H,10,USA,St. Petersburg
8,8,I,10,USA,St. Petersburg
9,9,J,100,Germany,Potsdam


It's a beauty! 

In [13]:
d.pair_features

{(1,
  2): {'1': array([0.        , 0.        , 0.        , 0.33333333, 0.33333333,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 1.        ]), '2': array([0.        , 0.        , 0.        , 0.33333333, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]), '3': array([0.        , 0.        , 0.        , 0.33333333, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]), '10': array([0.        , 0.        , 0.        , 0.        , 0.66666667,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ])},
 (2,
  2): {'1': array([0.        , 0.        , 0.        , 0.33333333, 0.33333333,
         0.        , 0.        , 0.        , 0.       